# Build Sample Dataset

## Setup

In [5]:
# Setup and Imports
import json
import pandas as pd

## Load Datasets

In [6]:
prm800k = pd.read_csv("dataset/prm800k-test-valid-data.csv")
prm800k

,labeler,timestamp,generation,is_quality_control_question,is_initial_screening_question,question,label,total_steps,total_steps_have_neg_1,steps_neg_1,total_neg_1_sequence_from_last_step,total_steps_have_zero,steps_zero,total_steps_have_pos_1_or_human,steps_pos_1_or_human,total_steps_have_combination,finish_reason
0,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-30T14:37:13.296218,NaN,False,False,{'problem': 'There are an infinite number of v...,"{'steps': [{'completions': [{'text': ""Let's se...",34,22,"[2, 3, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...",3,14,"[2, 3, 4, 6, 7, 9, 11, 13, 18, 20, 24, 26, 27,...",34,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",11,solution
1,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-30T13:26:58.414691,NaN,False,False,{'problem': 'When rolling a certain unfair six...,"{'steps': [{'completions': [{'text': ""Well, le...",35,18,"[2, 3, 4, 6, 8, 9, 12, 13, 14, 17, 18, 19, 20,...",0,11,"[3, 4, 8, 9, 12, 13, 15, 18, 21, 28, 31]",35,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",10,solution
2,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-31T14:39:30.588403,NaN,False,False,{'problem': 'Find all solutions to\n\\[\\sin \...,"{'steps': [{'completions': [{'text': ""Let's us...",34,13,"[1, 2, 6, 7, 8, 14, 16, 18, 29, 30, 31, 32, 34]",1,7,"[2, 7, 8, 14, 16, 33, 34]",34,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",6,solution
3,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-29T07:48:01.714041,NaN,False,False,{'problem': 'The solutions of the equation $z^...,{'steps': [{'completions': [{'text': 'There is...,40,17,"[2, 6, 8, 9, 11, 12, 16, 17, 18, 20, 25, 30, 3...",0,8,"[2, 3, 16, 17, 18, 20, 30, 40]",40,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",6,solution
4,e90a38f3-3135-4465-87af-3e6322e3d772,2022-07-22T20:02:50.866783,NaN,False,False,{'problem': 'A sequence $(a_n)$ is defined as ...,"{'steps': [{'completions': [{'text': ""So we're...",36,9,"[5, 9, 10, 11, 12, 13, 14, 15, 16]",0,8,"[5, 6, 9, 11, 12, 14, 16, 35]",36,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",6,solution
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,a2e09d3a-2da6-4094-89ca-e92125f6aefd,2022-11-11T23:53:32.446394,6.0,False,False,{'problem': 'Find the product of $6_8 \\cdot 7...,{'steps': [{'completions': [{'text': 'I notice...,15,0,[],0,0,[],15,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0,solution
538,2600b6ef-d904-43e8-ba92-de2fec65e275,2022-11-11T23:43:18.235408,6.0,False,False,"{'problem': 'Compute $\\dbinom{31}{28}$.', 'gr...",{'steps': [{'completions': [{'text': 'To compu...,6,1,[6],1,0,[],6,"[1, 2, 3, 4, 5, 6]",0,solution
539,fc08c519-6e30-4ca6-b223-37cc3bcb5879,2022-11-11T22:24:31.318743,6.0,False,False,"{'problem': ""Alice and Bob are playing a game....",{'steps': [{'completions': [{'text': 'I need t...,16,0,[],0,0,[],16,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0,solution
540,fc08c519-6e30-4ca6-b223-37cc3bcb5879,2022-11-11T22:19:53.088201,6.0,False,False,{'problem': 'In how many ways can $7$ people s...,{'steps': [{'completions': [{'text': 'This is ...,16,0,[],0,0,[],16,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0,solution


## Build Sample Dataset

### Functions

In [7]:
import json
import ast

def string_to_dict(input_string):
    input_string = input_string.strip()

    try:
        json_string = input_string.replace("'", '"')
        json_string = json_string.replace('True', 'true').replace('False', 'false').replace('None', 'null')
        return json.loads(json_string)
    except (json.JSONDecodeError, ValueError):
        pass
    
    try:
        return ast.literal_eval(input_string)
    except (ValueError, SyntaxError):
        pass
    
    try:
        if input_string.strip().startswith('{') and input_string.strip().endswith('}'):
            return eval(input_string)
    except:
        pass

    raise ValueError(f"Cannot parse string as dictionary: {input_string[:100]}...")

In [8]:
def process_data(df, function):
    results = []
    
    for index, row in df.iterrows():
        try:
            data = string_to_dict(row['label'])
            steps = data['steps']
            new_columns = function(steps)

            new_result = {
                **row.to_dict(),
                **new_columns,
            }
            results.append(new_result)
        
        except Exception as e:
            print(f"Error processing row {index}: {e}")
            continue
    
    # Create DataFrame from results
    results_df = pd.DataFrame(results)
    return results_df

### 1. Merge Completions in Each Steps to Index 0

In [9]:
def merge_completions_to_idx_0(steps):
    new_steps = []
    result = {
        "steps": new_steps
    }
    return result

In [10]:
df_option_1 = process_data(prm800k, merge_completions_to_idx_0)

In [11]:
df_option_1

,labeler,timestamp,generation,is_quality_control_question,is_initial_screening_question,question,label,total_steps,total_steps_have_neg_1,steps_neg_1,total_neg_1_sequence_from_last_step,total_steps_have_zero,steps_zero,total_steps_have_pos_1_or_human,steps_pos_1_or_human,total_steps_have_combination,finish_reason,steps
0,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-30T14:37:13.296218,NaN,False,False,{'problem': 'There are an infinite number of v...,"{'steps': [{'completions': [{'text': ""Let's se...",34,22,"[2, 3, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...",3,14,"[2, 3, 4, 6, 7, 9, 11, 13, 18, 20, 24, 26, 27,...",34,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",11,solution,[]
1,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-30T13:26:58.414691,NaN,False,False,{'problem': 'When rolling a certain unfair six...,"{'steps': [{'completions': [{'text': ""Well, le...",35,18,"[2, 3, 4, 6, 8, 9, 12, 13, 14, 17, 18, 19, 20,...",0,11,"[3, 4, 8, 9, 12, 13, 15, 18, 21, 28, 31]",35,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",10,solution,[]
2,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-31T14:39:30.588403,NaN,False,False,{'problem': 'Find all solutions to\n\\[\\sin \...,"{'steps': [{'completions': [{'text': ""Let's us...",34,13,"[1, 2, 6, 7, 8, 14, 16, 18, 29, 30, 31, 32, 34]",1,7,"[2, 7, 8, 14, 16, 33, 34]",34,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",6,solution,[]
3,debabc6d-f79c-4ee5-a9db-5e284390254c,2022-07-29T07:48:01.714041,NaN,False,False,{'problem': 'The solutions of the equation $z^...,{'steps': [{'completions': [{'text': 'There is...,40,17,"[2, 6, 8, 9, 11, 12, 16, 17, 18, 20, 25, 30, 3...",0,8,"[2, 3, 16, 17, 18, 20, 30, 40]",40,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",6,solution,[]
4,e90a38f3-3135-4465-87af-3e6322e3d772,2022-07-22T20:02:50.866783,NaN,False,False,{'problem': 'A sequence $(a_n)$ is defined as ...,"{'steps': [{'completions': [{'text': ""So we're...",36,9,"[5, 9, 10, 11, 12, 13, 14, 15, 16]",0,8,"[5, 6, 9, 11, 12, 14, 16, 35]",36,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",6,solution,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,a2e09d3a-2da6-4094-89ca-e92125f6aefd,2022-11-11T23:53:32.446394,6.0,False,False,{'problem': 'Find the product of $6_8 \\cdot 7...,{'steps': [{'completions': [{'text': 'I notice...,15,0,[],0,0,[],15,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0,solution,[]
538,2600b6ef-d904-43e8-ba92-de2fec65e275,2022-11-11T23:43:18.235408,6.0,False,False,"{'problem': 'Compute $\\dbinom{31}{28}$.', 'gr...",{'steps': [{'completions': [{'text': 'To compu...,6,1,[6],1,0,[],6,"[1, 2, 3, 4, 5, 6]",0,solution,[]
539,fc08c519-6e30-4ca6-b223-37cc3bcb5879,2022-11-11T22:24:31.318743,6.0,False,False,"{'problem': ""Alice and Bob are playing a game....",{'steps': [{'completions': [{'text': 'I need t...,16,0,[],0,0,[],16,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0,solution,[]
540,fc08c519-6e30-4ca6-b223-37cc3bcb5879,2022-11-11T22:19:53.088201,6.0,False,False,{'problem': 'In how many ways can $7$ people s...,{'steps': [{'completions': [{'text': 'This is ...,16,0,[],0,0,[],16,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...",0,solution,[]
